In [1]:
import numpy as np
from fancy_einsum import einsum
from einops import reduce, rearrange, repeat
from typing import Union, Optional, Callable
import torch as t
import torchvision
import utils

In [3]:
arr = np.load("numbers.npy")

In [5]:
utils.display_array_as_img(arr[0])

In [6]:
utils.display_array_as_img(rearrange(arr, 'n c h w->c h (n w)'))

In [18]:
stack = rearrange(arr[0:2], 'n c h w -> c (n h) w')
utils.display_array_as_img(repeat(stack, 'c h w -> c h (2 w)'))

In [20]:
utils.display_array_as_img(repeat(arr[0], 'c h w -> c (h 2) w'))

In [23]:
arr2 = rearrange(arr[0], "c h w -> h (c w)")
utils.display_array_as_img(arr2)

In [26]:
a = rearrange(arr, '(n1 n2) c h w -> c (n1 h) (n2 w)', n1=2)
utils.display_array_as_img(a)


In [29]:
da = utils.display_array_as_img

In [32]:
a = rearrange(arr, '(n1 n2) c h w -> c (n1 h) (n2 w)', n1=2)
da(a)

In [39]:
da(reduce(arr, 'n c h w -> h (n w)', 'sum'))

In [48]:
da(reduce(arr.astype(float), 'n c h w -> h w', 'min'))

In [70]:
col = rearrange(arr[0:2], 'n c h w -> c h (n w)')
da(rearrange(col, 'c (h2 h1) w -> c h1 (h2 w)', h2=2))

In [87]:
y = rearrange(arr, '(n1 n2) c h w -> c (n2 h) (n1 w)', n1=2)
x = rearrange(y, 'c h w -> c w h')
da(x)

In [90]:
a = rearrange(arr, '(n1 n2) c h w -> c (n1 h) (n2 w)', n1=2)
utils.display_array_as_img(reduce(a.astype(float), 'c (h h2) (w w2) -> c h w', 'mean', h2=2, w2=2))


In [92]:
a= reduce(arr, "b c (h 15) (w 15) -> b c h w", "max")
b= rearrange(arr2, "(b1 b2) c h w -> c (b1 h) (b2 w)", b1=2)
da(b)

EinopsError:  Error while processing rearrange-reduction pattern "(b1 b2) c h w -> c (b1 h) (b2 w)".
 Input tensor shape: (3, 150, 300). Additional info: {'b1': 2}.
 Expected 4 dimensions, got 3

In [111]:
def einsum_trace(mat: np.ndarray):
    return einsum('i i->',mat)

utils.test_trace(einsum_trace)

In [113]:
def einsum_mv(mat: np.ndarray, vec: np.ndarray):
    """
    Returns the same as `np.matmul`, when `mat` is a 2D array and `vec` is 1D.
    """
    return einsum('n m, m -> n', mat, vec)

utils.test_mv(einsum_mv)

In [115]:
def einsum_mm(mat1: np.ndarray, mat2: np.ndarray):
    """
    Returns the same as `np.matmul`, when `mat1` and `mat2` are both 2D arrays.
    """
    return einsum('a b, b c -> a c', mat1, mat2)

utils.test_mm(einsum_mm)

In [122]:
def einsum_inner(vec1, vec2):
    return einsum('a,a->', vec1, vec2)

def einsum_outer(vec1, vec2):
    """
    Returns the same as `np.outer`.
    """
    return einsum('a,b->a b', vec1, vec2)

utils.test_einsum_inner(einsum_inner)
utils.test_einsum_outer(einsum_outer)